# Model Building and Training

In this notebook we will train our model on mixed data through multiple inputs.

In [56]:
import tarfile
import pickle
import pandas as pd
import bz2
from datetime import datetime,timedelta
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import concatenate
import cv2
import os
import glob

Define functions:

In [9]:
def save_pickle(file_name,obj):
    with open(file_name, 'wb') as fout:
        pickle.dump(obj, fout)

def open_pickle(file_name):
    with open(file_name, 'rb') as handle:
        obj = pickle.load(handle)
    return obj

def get_np_array_from_tar_object(tar_extractfl):
    '''converts a buffer from a tar file in np.array'''
    return np.asarray(bytearray(tar_extractfl.read())
                      , dtype=np.uint8)


def load_sky_images(df,yr=None,tar=None):
    # initialize our images array (i.e., the house images themselves)
    images = []
    
    yr = '0'
    for img in df.values:
        yr_file = img[:4]
        if yr == yr_file:
            pass
        else:
            if tar:
                print("deleted tar")
                del tar
        
            yr = img[:4]
            file = f'data/Folsom_sky_images_{yr}.tar.bz2'
            print(yr,file)
            tar = tarfile.open(file)
        
        image = cv2.imdecode(get_np_array_from_tar_object(tar.extractfile(img)), 0)
        image = cv2.resize(image, (32, 32))
        images.append(image)
    
    return np.array(images)

### Read in Data:

In [11]:
model_data_dict = open_pickle('../data_rp/model_data_dict.pkl')

In next 3 cells open the images from the tar.bz2 files using the `load_sky_images()` function. This will be a numpy array of all of the images for each instance in the df. We aren't using the built in stream from directory function of keras due to the file type. The files are very large, so I would prefer not to unzip them on my machine

In [13]:
img_5_min = model_data_dict['df_5_min']['file']

In [21]:
sorted_img_5 = img_5_min.sort_values()

In [23]:
images_5_min = load_sky_images(sorted_img_5)

2014 data/Folsom_sky_images_2014.tar.bz2
deleted tar
2015 data/Folsom_sky_images_2015.tar.bz2
deleted tar
2016 data/Folsom_sky_images_2016.tar.bz2


In [27]:
# save_pickle('../data_rp/5_min_w_img.pkl',images_5_min)
# images_5_min.shape
images_5_min = open_pickle('../data_rp/5_min_w_img.pkl')

(10000, 32, 32)

In [106]:
train_ind = model_data_dict['df_5_min']['X_train'].index
test_ind = model_data_dict['df_5_min']['X_test'].index

In [90]:
sorted_img_5 = pd.DataFrame(sorted_img_5).reset_index()
# sorted_img_5['img_matrix'] = images_5_min

In [115]:
def get_index_of_img(train,test,sorted_img_dict):
    
    train_img,test_img = [],[]
    
    for i in train:
        train_img.append(sorted_img_5[sorted_img_5['index'] == i].index[0])
    
    for i in test:
        test_img.append(sorted_img_5[sorted_img_5['index'] == i].index[0])

    return train_img,test_img

def get_img_from_index(train_img_ind,test_img_ind,img_matrices):
    
    train_img,test_img = [],[]
    
    for i in train_img_ind:
        train_img.append(img_matrices[i])
    
    for i in test_img_ind:
        test_img.append(img_matrices[i])
    
    return np.array(train_img),np.array(test_img)

In [119]:
train_img_ind,test_img_ind = get_index_of_img(train_ind,test_ind,sorted_img_5)
trainImagesX,testImagesX = get_img_from_index(train_img_ind,test_img_ind,images_5_min)

### Process our image data and targets:

right now, we'll just start by scaling the images down from [0-255], to between [0-1].

In [117]:
# images_5_min = images_5_min / 255.0
train_img = train_img / 255.0
test_img = test_img / 255.0

In [74]:
# model_data_dict['df_5_min'].keys()

In [73]:
# find the largest irradiation in the training set and use it to
# scale our irradiation to the range [0, 1] (will lead to better
# training and convergence)
maxPrice = model_data_dict['df_5_min']['y_train'].max()
trainY = model_data_dict['df_5_min']['y_train'] / maxPrice
testY = model_data_dict['df_5_min']['y_test'] / maxPrice

### Build model:

The three parts of this section will be making adjustments to pre-trained CNN models, building an MLP, and then adding layers at the end to contatinate the results of these two branches of NN models, and then added fully connected and regression layers at the end.

So, let's break them out: 
1. [Update pre-trained CNN models for image data](#Update-pretrained-CNN-models)
2. [Build MLP for numeric/categorical data](#Build-MLP-for-numeric-and-categorical-data)
3. [Create end of multi-imput model](#Build-the-final-section-of-our-model)

#### Update pretrained CNN models

Starting with ResNet50, but will do the others next.

In [64]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

baseModelResNet = ResNet50(weights="imagenet", include_top=False,
                           input_tensor=Input(shape=(32, 32, 3)))

# baseModelIv3 = InceptionV3(weights="imagenet", include_top=False,
#                             input_tensor=Input(shape=(1,32, 32, 3)))

# baseModelResIv2 = InceptionResNetV2(weights="imagenet", include_top=False,
#                                       input_tensor=Input(shape=(32, 32, 3)))

In [66]:
chanDim = -1

# flatten the volume, then FC => RELU => BN => DROPOUT
headModel = baseModelResNet.output
# headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten()(headModel)
headModel = Dense(16)(headModel)
headModel = Activation("relu")(headModel)
headModel = BatchNormalization(axis=chanDim)(headModel)
headModel = Dropout(0.5)(headModel)
# apply another FC layer, this one to match the number of nodes
# coming out of the MLP
headModel = Dense(4)(headModel)
headModel = Activation("relu")(headModel)


baseModelResNet.trainable = False

cnn = Model(inputs=baseModelResNet.input, outputs=headModel)

In [118]:
# cnn.summary()

In [51]:
# # construct the head of the model that will be placed on top of the
# # the base model
# headModel = baseModel.output
# headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
# headModel = Flatten(name="flatten")(headModel)
# headModel = Dense(256, activation="relu")(headModel)
# headModel = Dropout(0.5)(headModel)
# headModel = Dense(len(config.CLASSES), activation="softmax")(headModel)
# # place the head FC model on top of the base model (this will become
# # the actual model we will train)
# model = Model(inputs=baseModel.input, outputs=headModel)
# # loop over all layers in the base model and freeze them so they will
# # *not* be updated during the training process
# for layer in baseModel.layers:
#     layer.trainable = False

#### Build MLP for numeric and categorical data

We're going to start with something super basic initially to see how this does.

In [28]:
def create_mlp(dim, regress=False):
    # define our MLP network
    model = Sequential()
    model.add(Dense(8, input_dim=dim, activation="relu"))
    model.add(Dense(4, activation="relu"))
    # check to see if the regression node should be added
    if regress:
        model.add(Dense(1, activation="linear"))
        # return our model
        
    return model

In [54]:
trainAttrX = model_data_dict['df_5_min']['X_train_p']
mlp = create_mlp(trainAttrX.shape[1], regress=False)

#### Build the final section of our model

Fully connected and with linear activation function for regression output.

In [63]:
# create the input to our final set of layers as the *output* of both
# the MLP and CNN
combinedInput = concatenate([mlp.output, cnn.output])

# our final FC layer head will have two dense layers, the final one
# being our regression head
x = Dense(4, activation="relu")(combinedInput)
x = Dense(1, activation="linear")(x)

# our final model will accept categorical/numerical data on the MLP
# input and images on the CNN input, outputting a single value (the
# predicted price of the house)
model = Model(inputs=[mlp.input, cnn.input], outputs=x)

In [120]:
testAttrX = model_data_dict['df_5_min']['X_test_p']

In [123]:
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)
# train the model
print("[INFO] training model...")
model.fit(
    x=[trainAttrX, trainImagesX], y=trainY,
    validation_data=([testAttrX, testImagesX], testY),
    epochs=200, batch_size=8)

[INFO] training model...
Epoch 1/200


ValueError: in user code:

    /Users/elena.morais/Documents/environments/reseach_paper/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /Users/elena.morais/Documents/environments/reseach_paper/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/elena.morais/Documents/environments/reseach_paper/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/elena.morais/Documents/environments/reseach_paper/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/elena.morais/Documents/environments/reseach_paper/lib/python3.7/site-packages/tensorfl

In [ ]:
# make predictions on the testing data
print("[INFO] predicting irridiation...")
preds = model.predict([testAttrX, testImagesX])

#### Create final layers of multi input model